In [2]:
from pyspark.sql.functions import *
from pyspark.sql import SQLContext

In [3]:
sqlsc = SQLContext(sc)

In [5]:
df= sqlsc.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/employees",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "employees",
    user="root",
    password="cloudera").load()

In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
df

DataFrame[emp_no: int, birth_date: date, first_name: string, last_name: string, gender: string, hire_date: date]

In [6]:
df.count()

300024

In [7]:
df.printSchema()

root
 |-- emp_no: integer (nullable = false)
 |-- birth_date: date (nullable = false)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- hire_date: date (nullable = false)



In [8]:
df.show(5)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|     M|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|     F|1985-11-21|
| 10003|1959-12-03|     Parto|  Bamford|     M|1986-08-28|
| 10004|1954-05-01| Chirstian|  Koblick|     M|1986-12-01|
| 10005|1955-01-21|   Kyoichi| Maliniak|     M|1989-09-12|
+------+----------+----------+---------+------+----------+
only showing top 5 rows



In [9]:
df.select("first_name","last_name").show(5)

+----------+---------+
|first_name|last_name|
+----------+---------+
|    Georgi|  Facello|
|   Bezalel|   Simmel|
|     Parto|  Bamford|
| Chirstian|  Koblick|
|   Kyoichi| Maliniak|
+----------+---------+
only showing top 5 rows



In [16]:
df.filter(df["gender"]=="F").select("emp_no","hire_date", "first_name").show(5)

+------+----------+----------+
|emp_no| hire_date|first_name|
+------+----------+----------+
| 10002|1985-11-21|   Bezalel|
| 10006|1989-06-02|    Anneke|
| 10007|1989-02-10|   Tzvetan|
| 10009|1985-02-18|    Sumant|
| 10010|1989-08-24| Duangkaew|
+------+----------+----------+
only showing top 5 rows



In [18]:
df.groupBy("gender").count().show(5)

+------+------+
|gender| count|
+------+------+
|     F|120051|
|     M|179973|
+------+------+



In [19]:
titles= sqlsc.read.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/employees",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "titles",
    user="root",
    password="cloudera").load()

In [20]:
titles.show(5)

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10001|Senior Engineer|1986-06-26|9999-01-01|
| 10002|          Staff|1996-08-03|9999-01-01|
| 10003|Senior Engineer|1995-12-03|9999-01-01|
| 10004|       Engineer|1986-12-01|1995-12-01|
| 10004|Senior Engineer|1995-12-01|9999-01-01|
+------+---------------+----------+----------+
only showing top 5 rows



In [21]:
titles.count()

443308

In [23]:
merge = df.join(titles,"emp_no")

In [26]:
merge.count()

443308

In [25]:
merge.printSchema()

root
 |-- emp_no: integer (nullable = false)
 |-- birth_date: date (nullable = false)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- hire_date: date (nullable = false)
 |-- title: string (nullable = false)
 |-- from_date: date (nullable = false)
 |-- to_date: date (nullable = true)



In [24]:
merge.show()

+------+----------+-----------+-----------+------+----------+----------------+----------+----------+
|emp_no|birth_date| first_name|  last_name|gender| hire_date|           title| from_date|   to_date|
+------+----------+-----------+-----------+------+----------+----------------+----------+----------+
| 10031|1959-01-27|    Karsten|     Joslin|     M|1991-09-01|        Engineer|1991-09-01|1998-09-01|
| 10031|1959-01-27|    Karsten|     Joslin|     M|1991-09-01| Senior Engineer|1998-09-01|9999-01-01|
| 10231|1963-07-21|    Shaowen|    Desikan|     F|1996-04-13|           Staff|1998-09-09|9999-01-01|
| 10431|1961-05-25|   Patricio|    Bugrara|     M|1987-10-08|    Senior Staff|1995-10-08|9999-01-01|
| 10431|1961-05-25|   Patricio|    Bugrara|     M|1987-10-08|           Staff|1987-10-08|1995-10-08|
| 10631|1955-05-17|  Inderjeet|  Chaudhuri|     M|1988-02-16|        Engineer|1994-10-18|9999-01-01|
| 10831|1953-10-11|      Toong|   Driscoll|     M|1996-06-21|Technique Leader|1998-02-22|99